In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from requests_html import HTML
import time

import pandas as pd

In [2]:
url = 'https://s.cafef.vn/Lich-su-giao-dich-BVH-1.chn?fbclid=IwAR0e98txe3qOw8SP_cTAVxXqeTN2CnuAiOnnLMzUXovyH-zJRZXVNBWU2sg'

In [3]:
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://s.cafef.vn/robots.txt')
rp.read()

In [4]:
row_id = []
for i in range(1, 21):
    if i%2: row_id.append(str(i).zfill(2) + '_')
    else: row_id.append(str(i).zfill(2) + '_alt')

In [5]:
if rp.can_fetch('*', url):
    file = open('bvh_data.csv', 'w', encoding='utf-8')
    file.write(f'Date\tOpen\tHigh\tLow\tClose\n')
    
    driver = webdriver.Chrome(executable_path='./chromedriver.exe')
    driver.get(url)
    html = HTML(html=driver.page_source)
    
    while True:
        for i in row_id:
            row = html.find('tr#ctl00_ContentPlaceHolder1_ctl03_rptData2_ctl' + i + 'itemTR', first=True)
            if row:
                date = row.find('td.Item_DateItem', first=True).text
                prices = row.find('td.Item_Price10')
                op, hi, lo, cl = prices[5].text, prices[6].text, prices[7].text, prices[1].text
                file.write(f'{date}\t{op}\t{hi}\t{hi}\t{cl}\n')
                
        button = driver.find_elements(By.LINK_TEXT, '>')
        if len(button) > 0:
            button[0].click()
            time.sleep(2)
        else:
            break
    
    file.close()

In [6]:
pd.read_csv('bvh_data.csv', sep='\t').head(5)

,Date,Open,High,Low,Close
0,16/12/2020,58.4,58.6,58.6,58.6
1,15/12/2020,58.6,59.4,59.4,58.2
2,14/12/2020,58.0,59.4,59.4,58.6
3,11/12/2020,56.7,57.5,57.5,57.5
4,10/12/2020,57.9,58.0,58.0,56.7
